### Overview of this notebook

* motivating example with modular building blocks
    * connecting distances, aligners, classifiers
* pairwise transformers - the "type" of time series distances and kernels
* time series alignment and alignment distances, e.g., time warping
* composition patterns for distances, kernels, aligners
* outlook, roadmap, opportunities

In [ ]:
import warnings

warnings.filterwarnings("ignore")

## 3.1 Motivating example

Rich component relationships between object types!

* many classifiers, regressors, clusterers use distances or kernels
* distances and kernels are often composite, e.g., sum-of-distance, independent distance
* TS distances are often based on scalar multivariate distances (e.g., Euclidean)
* TS distances are often based on alignment, TS aligners are an estimator type!
* aligners internally typically use scalar uni/multivariate distances

example:

* 1-nn using `sklearn` nearest neighbors
* with multivariate dynamic time warping distance, from `dtw-python` library 
* on multivariate `"mahalanobis"` distance from `scipy`
* in `sktime` compatible interface, constructed from custom components

so, conceptually:

* we build an sequence alignment algorithm (`dtw-python`) using `scipy` Mahalanobis dist
* we get the distance matrix computation from alignment algorithm
* we use that distance matrix in `sklearn` knn
* together this is a time series classifier!

In [ ]:
from sktime.alignment.dtw_python import AlignerDTWfromDist
from sktime.classification.distance_based import KNeighborsTimeSeriesClassifier
from sktime.dists_kernels.compose_from_align import DistFromAligner
from sktime.dists_kernels.scipy_dist import ScipyDist

# Mahalanobis distance on R^n
mahalanobis_dist = ScipyDist(metric="mahalanobis")  # uses scipy distances

# pairwise multivariate aligner from dtw-python with Mahalanobis distance
mw_aligner = AlignerDTWfromDist(mahalanobis_dist)  # uses dtw-python

# turning this into alignment distance on time series
dtw_dist = DistFromAligner(mw_aligner)  # interface mutation to distance

# and using this distance in a k-nn classifier
clf = KNeighborsTimeSeriesClassifier(distance=dtw_dist)  # uses sklearn knn

In [ ]:
clf.get_params()

what are all the objects in this chain?

* `ScipyDist` - pairwise distance between *scalars* - `transformer-pairwise` type
* `AlignerDtwFromDist` - time series alignment algorithm - `aligner` type
* `DistFromAligner`- pairwise distance between *time series* - `transformer-pairwise-panel` type
* `KNeighborsTimeSeriesClassifier` - time series classifier

In [ ]:
from sktime.registry import scitype

scitype(mw_aligner)  # prints the type of estimator (as a string)
# same for other components

let's go through these - we've already seen classifiers.

## 3.2 Time series distances and kernels - pairwise panel transformers

### 3.2.1 Distances, kernels - general interface

pairwise panel transformers produce one distance per pair of series in the panel:

In [ ]:
from sktime.datasets import load_osuleaf

# load an example time series panel in numpy mtype
X, _ = load_osuleaf(return_type="numpy3D")

X1 = X[:3]
X2 = X[5:10]

In [ ]:
# constructing the transformer
from sktime.dists_kernels import FlatDist
from sktime.dists_kernels.scipy_dist import ScipyDist

# paired Euclidean distances, over time points
eucl_dist = FlatDist(ScipyDist())

In [ ]:
X1.shape

In [ ]:
X2.shape

X1 is panel with 3 series
X2 is panel with 5 series

so a matrix of pairwise distances from X1 to X2 should have shape (3, 5)

In [ ]:
distmat = eucl_dist(X1, X2)

# alternatively, via the transform method
distmat = eucl_dist.transform(X1, X2)
distmat

In [ ]:
distmat.shape

call or `transform` with a single arg is the same as passing twice:

In [ ]:
distmat_symm = eucl_dist.transform(X1)
distmat_symm

pairwise panel transformers are `scikit-learn` / `scikit-base` interface compatible and composable, like everything else in `sktime`:

In [ ]:
eucl_dist.get_params()

### 3.2.2 Time series distances, kernels - composition

pairwise transformers can be composed in a number of ways:

* arithmetics, e.g., addition, multiplication - use dunder `+`, `*` etc, or `CombinedDistance`
* subset to one or multiple columns - use `my_dist[colnames]` dunder
* sum or aggregate over univariate distance in multivariate panel, using `IndepDist` (also known as "independent distance")
* compose with series-to-series transformers - use `*` dunder or `make_pipeline`

In [ ]:
from sktime.datasets import load_basic_motions

# load an example time series panel in numpy mtype
X, _ = load_basic_motions(return_type="numpy3D")
X = X[:3]
X.shape

In [ ]:
# example 1: variable subsetting and arithmetic combinations

# we define *two* distances now
from sktime.dists_kernels import FlatDist, ScipyDist

# Euclidean distance (on flattened time series)
eucl_dist = FlatDist(ScipyDist())
# Mahalanobis distance (on flattened time series)
cos_dist = FlatDist(ScipyDist(metric="cosine"))

# arithmetic product of:
# * the Euclidean distance on gyrometer 2 time series
# * the Mahalanobis distance on accelerometer 3 time series
prod_dist_42 = eucl_dist[4] * cos_dist[2]
prod_dist_42

In [ ]:
prod_dist_42(X)

In [ ]:
# example 2: independent dynamic time warping distance
from sktime.alignment.dtw_python import AlignerDTW
from sktime.dists_kernels.compose_from_align import DistFromAligner
from sktime.dists_kernels.indep import IndepDist

# dynamic time warping distance - this is multivariate
dtw_dist = DistFromAligner(AlignerDTW())

# independent distance - by default IndepDist sums over univariate distances
indep_dtw_dist = IndepDist(dtw_dist)

# that is, this distance is arithmetic sum of
# * DTW distance on accelerometer 1 time series
# * DTW distance on accelerometer 2 time series
# * DTW distance on accelerometer 3 time series
# * DTW distance on gyrometer 1 time series
# * DTW distance on gyrometer 2 time series
# * DTW distance on gyrometer 3 time series

In [ ]:
indep_dtw_dist(X)

In [ ]:
# example 3: dynamic time warping distance on first differences
from sktime.transformations.series.difference import Differencer

diff_dtw_distance = Differencer() * dtw_dist

In [ ]:
diff_dtw_distance(X)

some combinations may be available as efficient `numba` based distances.

E.g., difference-then-dtw is available as the "fixed" `sktime` native implementation
`DtwDist(derivative=True)` in `sktime.dists_kernels.dtw`.

### 3.3 pairwise tabular transformers

### 3.3.1 pairwise tabular transformers - general interface

pairwise tabular transformers transform pairs of ordinary tabular data, e.g., plain `pd.DataFrame`

produce one distance per pair of rows

In [ ]:
from sktime.datatypes import get_examples

# we retrieve some DataFrame examples
X_tabular = get_examples("pd.DataFrame", "Series")[1]
X2_tabular = get_examples("pd.DataFrame", "Series")[1][0:3]

In [ ]:
# just an ordinary DataFrame, no time series
X_tabular

In [ ]:
X2_tabular

example: pairwise Euclidean distance between rows

In [ ]:
# constructing the transformer
from sktime.dists_kernels import ScipyDist

# mean of paired Euclidean distances
my_tabular_dist = ScipyDist(metric="euclidean")

In [ ]:
# obtain matrix of distances between each pair of rows in X_tabular, X2_tabular
my_tabular_dist(X_tabular, X2_tabular)

In [ ]:
# alternative call with transform:
my_tabular_dist.transform(X_tabular, X2_tabular)

In [ ]:
# as with pairwise panel transformers, one arg means second is the same
my_tabular_dist(X_tabular)

### 3.3.2 constructing pairwise time series transformers from tabular ones

"simple" time series distances can be obtained directly from tabular transformers:

* flattening the time series to tabular, and then computing the distance - `FlatDist`
* aggregating the tabular distance matrix, from two individual time series - `AggrDist`

these are important "baseline" distances!

Both can be used on `sktime` pairwise transformers and `sklearn` pairwise transformers.

the classes are called "dist" but all apply to kernels.

In [ ]:
from sktime.datasets import load_basic_motions

# load an example time series panel in numpy mtype
X, _ = load_basic_motions(return_type="numpy3D")
X = X[:3]
X.shape

In [ ]:
# example 1: flat Gaussian RBF kernel between time series
from sklearn.gaussian_process.kernels import RBF
from sktime.dists_kernels import FlatDist

flat_gaussian_tskernel = FlatDist(RBF(length_scale=10))
flat_gaussian_tskernel.get_params()

In [ ]:
flat_gaussian_tskernel(X)

In [ ]:
# example 2: pairwise cosine distance - we've already seen FlatDist a couple times
from sktime.dists_kernels import FlatDist, ScipyDist

mean_cos_tsdist = FlatDist(ScipyDist(metric="cosine"))
mean_cos_tsdist.get_params()

In [ ]:
mean_cos_tsdist(X)

## 3.4 alignment algorithms, aka aligners

* "aligners" find a new index set for 2 or more time series so they become "similar"
* new index set being a non-linear reparameterization of the old index sets
* often, aligners also produce an overall distance between the two series

### 3.4.1 aligners - general interface

aligner methods:

* `fit` - computes alignment
* `get_alignment` - returns reparametrized indices, also called "alignment path"
* `get_aligned` returns reparametrized series
* `get_distance` returns distance between the two aligned series - only available if `"capability:get_distance"`

let's try to align two leaf contours from osuleaf!

In [ ]:
from sktime.datasets import load_osuleaf

# load an example time series panel in numpy mtype
X, _ = load_osuleaf(return_type="pd-multiindex")

X1 = X.loc[0]  # leaf 0
X2 = X.loc[1]  # leaf 1

In [ ]:
from sktime.utils.plotting import plot_series

plot_series(X1, X2, labels=["leaf_1", "leaf_2"])

In [ ]:
from sktime.alignment.dtw_python import AlignerDTW

# use dtw-python package for aligning
# simple univariate alignment algorithm with default params
aligner = AlignerDTW()

In [ ]:
aligner.fit([X1, X2])  # series to align need to be passed as list

In [ ]:
# alignment path
aligner.get_alignment()

# this aligns, e.g.:
# from row "2": aligns index 0 in X1 with index 2 of X2
# from row "664": aligns index 424 in X1 with index 423 of X2

In [ ]:
# obtain the aligned versions of the two series
X1_al, X2_al = aligner.get_aligned()

In [ ]:
from sktime.utils.plotting import plot_series

plot_series(X1_al.reset_index(drop=True), X2_al.reset_index(drop=True), labels=["leaf_1", "leaf_2"])

the DTW aligner has a "distance" implemented

intuitively, it is a distance that sums distance after aligning, and the amount of stretch:

In [ ]:
# the AlignerDTW class (based on dtw-python) doesn't just align, it also produces a distance
aligner.get_tags()

In [ ]:
# this is the distance between the two time series we aligned
aligner.get_distance()

### 3.4.2 alignment based time series distances

the `DistFromAligner` wrapper simply computes distance per pair of aligned series.

This turns any aligner into a time series distance:

In [ ]:
from sktime.alignment.dtw_python import AlignerDTW
from sktime.dists_kernels.compose_from_align import DistFromAligner

# dynamic time warping distance - this is multivariate
dtw_dist = DistFromAligner(AlignerDTW())

In [ ]:
from sktime.datasets import load_osuleaf

# load an example time series panel in numpy mtype
X, _ = load_osuleaf(return_type="numpy3D")

X1 = X[:3]
X2 = X[5:10]

In [ ]:
dtw_distmat = dtw_dist(X1, X2)
dtw_distmat

In [ ]:
dtw_distmat.shape

## 3.5 revisiting the initial example

In [ ]:
from sktime.alignment.dtw_python import AlignerDTWfromDist
from sktime.classification.distance_based import KNeighborsTimeSeriesClassifier
from sktime.dists_kernels.compose_from_align import DistFromAligner
from sktime.dists_kernels.scipy_dist import ScipyDist

# Mahalanobis distance on R^n
mahalanobis_dist = ScipyDist(metric="mahalanobis")  # uses scipy distances

# pairwise multivariate aligner from dtw-python with Mahalanobis distance
mw_aligner = AlignerDTWfromDist(mahalanobis_dist)  # uses dtw-python

# turning this into alignment distance on time series
dtw_dist = DistFromAligner(mw_aligner)  # interface mutation to distance

# and using this distance in a k-nn classifier
clf = KNeighborsTimeSeriesClassifier(distance=dtw_dist)  # uses sklearn knn

In [ ]:
clf

* we build an sequence alignment algorithm (`dtw-python`) using `scipy` Mahalanobis dist
* we get the distance matrix computation from alignment algorithm
* we use that distance matrix in `sklearn` knn
* together this is a time series classifier!

## 3.6 Searching for distances, kernels, transformers

As with all `sktime` objects, we can use the `registry.all_estimators` utility to display all transformers in `sktime`.

The relevant scitypes are:

* `"transformer-pairwise"` for all pairwise transformers on tabular data
* `"transformer-panel"` for all pairwise transformers on panel data
* `"aligner"` for all time series aligners
* `"transformer"` for all transformers, these can be composed with all the aboev

In [ ]:
from sktime.registry import all_estimators

In [ ]:
# listing all pairwise panel transformers - distances, kernels on time series
all_estimators("transformer-pairwise-panel", as_dataframe=True)

In [ ]:
# listing all pairwise (tabular) transformers - distances, kernels on vectors/df-rows
all_estimators("transformer-pairwise", as_dataframe=True)

In [ ]:
# listing all alignment algorithms that can produce distances
all_estimators("aligner", as_dataframe=True, filter_tags={"capability:distance": True})

## 3.7 Outlook, roadmap

* implementing estimators - distances, classifiers, etc
* backend optimizations - `numba`, distributed/parallel
* sequence-to-sequence regression, classification
* further maturing the time series alignment module

join and contribute!

## 3.8 Summary

* `sktime` - modular framework for learning with time series
* panel data = collections of time series - tasks classification, regression, clustering
* build flexible pipelines with transformers, tune via grid search etc
* panel estimators typically rely on time series distances, kernels, aligners
* TS distances, kernels, aligners can also be constructed in modular, flexible way
* all objects above are first-class citiziens with `sklearn`-like interface!